In [10]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.stats import norm
from scipy.optimize import fsolve

In [ ]:
# Create a dropdown widget where you can select a company ticker
import ipywidgets as widgets
from IPython.display import display

# Comprehensive list of companies across risk categories
# Format: 'TICKER - Company Name'
company_options = [
    # LOW RISK COMPANIES (Large Cap, Stable)
    'AAPL - Apple Inc.',
    'MSFT - Microsoft Corporation',
    'GOOGL - Alphabet Inc.',
    'JNJ - Johnson & Johnson',
    'PG - Procter & Gamble Co.',
    'KO - The Coca-Cola Company',
    'PEP - PepsiCo Inc.',
    'WMT - Walmart Inc.',
    'V - Visa Inc.',
    'MA - Mastercard Inc.',
    'HD - The Home Depot Inc.',
    'UNH - UnitedHealth Group Inc.',
    'JPM - JPMorgan Chase & Co.',
    'BAC - Bank of America Corp.',
    'DIS - The Walt Disney Company',
    'NVDA - NVIDIA Corporation',
    'TSLA - Tesla Inc.',
    'BRK-B - Berkshire Hathaway Inc.',
    'XOM - Exxon Mobil Corporation',
    'CVX - Chevron Corporation',
    
    # MEDIUM RISK COMPANIES (Mid Cap, Cyclical)
    'F - Ford Motor Company',
    'GM - General Motors Company',
    'UAL - United Airlines Holdings Inc.',
    'AAL - American Airlines Group Inc.',
    'DAL - Delta Air Lines Inc.',
    'NCLH - Norwegian Cruise Line Holdings',
    'RCL - Royal Caribbean Cruises Ltd.',
    'CCL - Carnival Corporation',
    'MGM - MGM Resorts International',
    'LVS - Las Vegas Sands Corp.',
    'WYNN - Wynn Resorts Ltd.',
    'MAR - Marriott International Inc.',
    'HLT - Hilton Worldwide Holdings Inc.',
    'IHG - InterContinental Hotels Group',
    'X - United States Steel Corporation',
    'CLF - Cleveland-Cliffs Inc.',
    'AA - Alcoa Corporation',
    'FCX - Freeport-McMoRan Inc.',
    'NEM - Newmont Corporation',
    'APA - APA Corporation',
    
    # HIGH RISK COMPANIES (Small Cap, Volatile, High Debt)
    'AMC - AMC Entertainment Holdings Inc.',
    'GME - GameStop Corp.',
    'BBBY - Bed Bath & Beyond Inc.',
    'EXPR - Express Inc.',
    'SEARS - Sears Holdings Corporation',
    'JCP - J.C. Penney Company Inc.',
    'HTZ - Hertz Global Holdings Inc.',
    'PTON - Peloton Interactive Inc.',
    'ZOOM - Zoom Video Communications Inc.',
    'BYND - Beyond Meat Inc.',
    'PLUG - Plug Power Inc.',
    'NKLA - Nikola Corporation',
    'RIDE - Lordstown Motors Corp.',
    'LCID - Lucid Group Inc.',
    'RIVN - Rivian Automotive Inc.',
    'SPCE - Virgin Galactic Holdings Inc.',
    'HOOD - Robinhood Markets Inc.',
    'COIN - Coinbase Global Inc.',
    'ROKU - Roku Inc.',
    'SNAP - Snap Inc.',
    'TWTR - Twitter Inc.',
    'UBER - Uber Technologies Inc.',
    'LYFT - Lyft Inc.',
    'DASH - DoorDash Inc.',
    'SNOW - Snowflake Inc.',
    'PLTR - Palantir Technologies Inc.',
    'WISH - ContextLogic Inc.',
    'DKNG - DraftKings Inc.',
    'PENN - PENN Entertainment Inc.',
    'FUBO - fuboTV Inc.'
]

# Create a dropdown widget for company selection
company_dropdown = widgets.Dropdown(
    options=company_options,
    value='MSFT - Microsoft Corporation',  # Default selection
    description='Company:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# Create a button to update the selection
update_button = widgets.Button(
    description='Select Company',
    button_style='success'
)

# Create an output widget to show the selected company
output = widgets.Output()

def on_button_click(b):
    global selected_ticker, selected_company_name
    # Extract ticker from the dropdown selection (format: 'TICKER - Company Name')
    selection = company_dropdown.value
    selected_ticker = selection.split(' - ')[0]
    selected_company_name = selection.split(' - ')[1]
    
    with output:
        output.clear_output()
        print(f"✅ Selected: {selected_ticker} - {selected_company_name}")

update_button.on_click(on_button_click)

# Display the widgets
display(company_dropdown, update_button, output)

# Initialize with default selection
selected_ticker = 'MSFT'
selected_company_name = 'Microsoft Corporation'
print(f"Default selection: {selected_ticker} - {selected_company_name}")
print("👆 Choose a company from the dropdown and click 'Select Company' button")
print("\n📊 Risk Categories Included:")
print("🟢 Low Risk: Large cap, stable companies (AAPL, MSFT, JNJ, etc.)")
print("🟡 Medium Risk: Cyclical, travel, commodities (UAL, F, MGM, etc.)")
print("🔴 High Risk: Small cap, volatile, high debt (AMC, GME, PTON, etc.)")

Dropdown(description='Company:', index=1, layout=Layout(width='400px'), options=('AAPL - Apple Inc.', 'MSFT - …

Button(button_style='success', description='Select Company', style=ButtonStyle())

Output()

Default selection: MSFT - Microsoft Corporation
👆 Choose a company from the dropdown and click 'Select Company' button

📊 Risk Categories Included:
🟢 Low Risk: Large cap, stable companies (AAPL, MSFT, JNJ, etc.)
🟡 Medium Risk: Cyclical, travel, commodities (UAL, F, MGM, etc.)
🔴 High Risk: Small cap, volatile, high debt (AMC, GME, PTON, etc.)


In [12]:
# --- 1. DATA GATHERING ---
# Use the selected ticker from the widget above
ticker_symbol = selected_ticker
company = yf.Ticker(ticker_symbol)

print(f"🎯 Analyzing: {ticker_symbol}")
print("-" * 30)

# Get Market Capitalization (Market Value of Equity, E)
E = company.info['marketCap']

# Get Total Liabilities (Proxy for Face Value of Debt, K)
balance_sheet = company.balance_sheet

# Try to find Total Liabilities - check what's available first
print("Available balance sheet items:")
print(balance_sheet.index.tolist())

# Use the correct name (try multiple options for different companies)
if 'Total Liabilities Net Minority Interest' in balance_sheet.index:
    K = balance_sheet.loc['Total Liabilities Net Minority Interest'].iloc[0]
elif 'Total Liab' in balance_sheet.index:
    K = balance_sheet.loc['Total Liab'].iloc[0]
elif 'Total Debt' in balance_sheet.index:
    K = balance_sheet.loc['Total Debt'].iloc[0]
else:
    # Use total debt from company info as fallback
    K = company.info.get('totalDebt', E * 0.3)  # Assume 30% debt if not available
    print(f"⚠️  Using fallback debt estimate")

# Get Historical Stock Prices to Calculate Equity Volatility (sigma_E)
hist = company.history(period="1y")
log_returns = np.log(hist['Close'] / hist['Close'].shift(1))
sigma_E = log_returns.std() * np.sqrt(252) 

# Define Model Parameters
T = 1.0 
r = 0.045 # Example rate: 4.5%

print(f"\n--- Input Data for {ticker_symbol} ---")
print(f"Market Value of Equity (E): ${E:,.2f}")
print(f"Face Value of Debt (K):     ${K:,.2f}")
print(f"Annualized Equity Volatility (σ_E): {sigma_E:.2%}")
print(f"Time to Maturity (T): {T} year")
print(f"Risk-Free Rate (r): {r:.2%}")
print("-" * 35)

🎯 Analyzing: DAL
------------------------------
Available balance sheet items:
['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued', 'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital', 'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations', 'Common Stock Equity', 'Total Capitalization', 'Total Equity Gross Minority Interest', 'Stockholders Equity', 'Gains Losses Not Affecting Retained Earnings', 'Other Equity Adjustments', 'Treasury Stock', 'Retained Earnings', 'Additional Paid In Capital', 'Capital Stock', 'Common Stock', 'Total Liabilities Net Minority Interest', 'Total Non Current Liabilities Net Minority Interest', 'Other Non Current Liabilities', 'Employee Benefits', 'Non Current Pension And Other Postretirement Benefit Plans', 'Non Current Deferred Liabilities', 'Non Current Deferred Revenue', 'Non Current Deferred Taxes Liabilities', 'Long Term Debt And Capital Lease Obligation', 'Long Term Capital Lease Obligation', 'Long Term Debt

In [13]:
# --- 2. ITERATIVE SOLVER FOR V_A and sigma_A ---
def solve_merton_model(E, K, T, r, sigma_E):
    V_A = E + K 
    sigma_A = sigma_E * (E / (E + K))

    def equations(variables):
        v, s = variables
        if v <= 0 or s <= 0:
            return [1e6, 1e6]
        d1 = (np.log(v / K) + (r + 0.5 * s**2) * T) / (s * np.sqrt(T))
        d2 = d1 - s * np.sqrt(T)
        eq1 = E - (v * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2))
        eq2 = sigma_E - ((v / E) * norm.cdf(d1) * s)
        return [eq1, eq2]

    solution, _, _, _ = fsolve(equations, [V_A, sigma_A], full_output=True)
    return solution[0], solution[1]

V_A, sigma_A = solve_merton_model(E, K, T, r, sigma_E)

print(f"--- Solved Model Parameters ---")
print(f"Market Value of Assets (V_A): ${V_A:,.2f}")
print(f"Asset Volatility (σ_A): {sigma_A:.2%}")
print("-" * 35)

--- Solved Model Parameters ---
Market Value of Assets (V_A): $96,366,811,428.99
Asset Volatility (σ_A): 19.85%
-----------------------------------


In [14]:
# --- 3. CALCULATING CREDIT RISK METRICS ---
d1 = (np.log(V_A / K) + (r + 0.5 * sigma_A**2) * T) / (sigma_A * np.sqrt(T))
d2 = d1 - sigma_A * np.sqrt(T)

# Risk-Neutral Probability of Default
probability_of_default = norm.cdf(-d2)

# Market Value of Debt (F)
F_direct = V_A * norm.cdf(-d1) + K * np.exp(-r * T) * norm.cdf(d2)

# Yield to Maturity and Credit Spread
yield_to_maturity = - (1/T) * np.log(F_direct / K)
credit_spread = yield_to_maturity - r

print(f"--- Credit Risk Analysis ---")
print(f"Distance to Default (d2): {d2:.4f}")
print(f"Risk-Neutral Probability of Default: {probability_of_default:.4%}")
print(f"Market Value of Debt (Merton): ${F_direct:,.2f}")
print(f"Implied Yield on Debt: {yield_to_maturity:.2%}")
print(f"Credit Spread: {credit_spread*10000:.2f} basis points")

--- Credit Risk Analysis ---
Distance to Default (d2): 2.5077
Risk-Neutral Probability of Default: 0.6075%
Market Value of Debt (Merton): $57,414,318,373.95
Implied Yield on Debt: 4.54%
Credit Spread: 3.67 basis points
